# Lab 3 Extra: Dimensionality Reduction using Gaussian Processes
### Gaussian Process Summer School 2018
_written by Wil Ward, adapted from notebooks by Max Zwiessele and [Neil Lawrence](http://inverseprobability.com/)_

This lab focusses on using Gaussian processes for unsupervised analysis. We will apply dimensionality reduction to a dataset of images and compare it to Gaussian process latent variable modeling.

Extra labs are for you to explore in your own time, giving details of other uses of Gaussian processes not covered in the summer school. Answers for extra labs will be made available _after_ the summer school.

## 1. Getting Started

We import all the relevant packages, as in previous lab sessions, and create a utility for plotting our models (note this is different to the `plot_gp()` function used in previous labs).

In [ ]:
# Support for maths
import numpy as np
# Plotting tools
from matplotlib import pyplot as plt
# we use the following for plotting figures in jupyter
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# GPy: Gaussian processes library
import GPy

In [ ]:
# We will to order our colours for different classes (colourblind friendly)
colours = ["#B35806", "#F1A340", "#FEE0B6", "#D8DAEB", "#998EC3", "#542788"]#, "#6DDA4C", "#85831F", "#B36A29", "#CF4E4A"]

def plot_model(X, labels, which_dims=None):
    # Prepare dimensions of data
    X = X[:,:] if which_dims is None else X[:,which_dims]
    # Unique labels
    ulabs = []
    for lab in labels:
        if lab not in ulabs:
            ulabs.append(lab)
    # Prepare figure env.
    plt.figure()
    for i, lab in enumerate(ulabs):
        plt.scatter(*X[labels==lab].T, marker='o', color=colours[i], label=lab)

For this lab, we will also use a dataset of handwritten numbers, $0,\ldots,9$. We will use a subset of the digits in the lab but this can be extended to use all of them desired.

In [ ]:
GPy.util.datasets.authorize_download = lambda x: True # This gives GPy permission to download the dataset

# We only select these digits to work with
these_digits = [0, 1, 2, 6, 7, 9]
# Download the data, cache it locally and pass to variable `data`.
data = GPy.util.datasets.decampos_digits(which_digits=these_digits)

print("\nData keys:")
print(data.keys())

print("\nCitation:")
print(data['citation'])

print("\nInfo:")
print(data['info'])

Note we take our data, which exists only as the observed digits as `16x16` images. Each one has a corresponding label, indicating the number shown in the image.

In [ ]:
y = data['Y']
labels = data['str_lbls'].flatten()

print(y.shape)
print(labels.shape)

We will plot a random selection of the digits.

In [ ]:
f, axs = plt.subplots(3,3,figsize=(14,14))
for i,k in enumerate(np.random.randint(0, 329, size=(9,1))):
    axs[int(np.floor(i/3)), i%3].matshow(np.reshape(y[k,:],(16,16)))

## 2. Principal Component Analysis

Principal component analysis (PCA) finds a rotation of the observed outputs, such that the rotated principal component (PC) space maximises the variance of the data observed, sorted from most to least important. Here, we define importance as the variablity of a corresponding PC.

In [ ]:
# We create a PCA class with the digits dataset
p = GPy.util.pca.PCA(y)

p.plot_fracs(20); # We plot the first 20 eigenvalue fractions

In [ ]:
p.plot_2d(y, labels=labels, colors=colours) # We plot each digit according to its first two PCs
plt.legend();

We can see here that we can distinguish populations for most classes, though there is no necessarily a clear distinction for digits $1$ and $2$.

## 3. Gaussian Process Latent Variable Model
The Gaussian process latent variable model (GP-LVM) embeds a PCA into a Gaussian process framework, where the latent inputs $\mathbf{X}$ are learnt as hyperparameters and the mapping variables  $\mathbf{W}$ are integrated out. The advantage of this interpretation is that it allows PCA to be generalised in a non-linear way by replacing the resulting _linear_ covariance with a non-linear covariance.

We can see how GP-LVM is equivalent to PCA using an automatic relevance determination (ARD) linear kernel. We will define the latent space with $D=4$ dimensions. We can use `GPy` and its `GPLVM` model to perform this:

In [ ]:
# ARD linear kernel
k = GPy.kern.Linear(4, ARD=True)

# GPLVM with 4D latent space
m = GPy.models.GPLVM(y, input_dim=4, kernel=k)

# Optimise model
m.optimize(messages=1, max_iters=1000) 
# Preview our model
m

We plot the contribution of ARD in our respective latent dimensions

In [ ]:
m.kern.plot_ARD();

We can now compare the results of the linear ARD kernel with the PCA solution:

In [ ]:
plot_model(m.X, labels, m.linear.variances.argsort()[-2:])
plt.legend();

As we can see, the solution with a linear kernel is the same as that of the PCA, with some minor rotational changes. For the sake of time, the solution was only run for a maximum of 1000 iterations, though it converged before this. For the linear covariance, the latent points can be optimised with an eigenvalue problem but in the general case, particularly for non-linear covariance functions, gradient-based optimisation is needed.

### Exercise

(a) Try performing the GP-LVM with a non-linear covariance function, for example with the `RBF` kernel. How does the non-linear model differ from the linear one? Are there digits that the GP-LVM with non-linear covariance can seperate but that PCA is not able to?

(b) Try modifying the covariance function using additive or multiplicative combinations. How does this affect the result?

## 4. Bayesian GP-LVM

In GP-LVM, we used a point estimate of the distribution of the latent inputs $\mathbf{X}$. The estimate is derived through maximum likelihood or through a maximum _a posteriori_ (MAP) approach. Ideally, we would like to estimate a distribution over the input $\mathbf{X}$. In the _Bayesian_ GP-LVM, we approximate the true distribution $p(\mathbf{X}|\mathbf{y})$ using a variational approximation $q(\mathbf{X})$ and then integrating $\mathbf{X}$ out.

Approimating the posterior in this way allows us to optimise a lower bound on the marginal likelihood. Handling the uncertainty in a principled way allows the model to make an assessment of whether a particular latent dimension is required, or the variation is better explained by noise. This allows the algorithm to "_switch off_" latent dimensions. This switching off can be time consuming, and may take a while to optimise, so we use a sparse approximation.

We assume a 5-D space for the latent inputs, and set the number of inducing points for our sparse approximation to $M=25$.

In [ ]:
k = GPy.kern.RBF(5, ARD=True) # Non-linear ARD kernel

# Bayesian GPLVM model
m = GPy.models.BayesianGPLVM(y, input_dim=5, kernel=k, num_inducing=25)

# Optimise
m.optimize(messages=1, max_iters=2500)

# Preview the model
m

As before, we plot the contribution of ARD in our respective latent dimensions

In [ ]:
m.kern.plot_ARD();

And plot the two most _important_ dimensions of the means of the latent inputs:

In [ ]:
plot_model(m.X.mean, labels, m.rbf.lengthscale.argsort()[:2])
plt.legend();

### Exercise

How does the Bayesian GP-LVM compare with the standard GP-LVM model?